# `NLopt` with autodiff and numerical gradients
Presented by Chiyoung Ahn (https://github.com/chiyahn)

This notebook demonstrates how nonlinear optimization problems in `NLopt` can be solved without the need of specifying analytic formulae for gradients by autodifferentiation or numerical gradients.

In [ ]:
] add NLopt BenchmarkTools ForwardDiff NLSolversBase

In [15]:
using NLopt, BenchmarkTools, ForwardDiff, NLSolversBase

## Using vanilla `NLopt`

### Nonlinear optimization without nonlinear constraints:

First, define the objective function `f` and the corresponding gradient `g!`. In `NLopt`, evaluation of `f` and execution of `g!` take place in the same time:

In [16]:
# call g! then return f(x)
function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        grad[1] = -2*x[1]*(x[1]^2 + x[2]^2)
        grad[2] = -2*x[2]*(x[1]^2 + x[2]^2)
    end
    return -(x[1]^2 + x[2]^2)
end

fg! (generic function with 1 method)

and the corresponding optimization problem:

In [17]:
opt = Opt(:LD_LBFGS, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-1.0, -1.0]) # find `x` above -2.0
upper_bounds!(opt, [2.0, 2.0]) # find `x` below 2.0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

and solve it!

In [18]:
(minf,minx,ret) = @btime optimize(opt, [1.0, 1.0])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  6.509 ms (20 allocations: 960 bytes)
got -8.0 at [2.0, 2.0] after 2 iterations (returned SUCCESS)


### Nonlinear optimization with nonlinear constraints:

Define `fg!` first:

In [19]:
function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end
    return sqrt(x[2]) # f(x)
end

fg! (generic function with 1 method)

and the corresponding optimization problem:

In [20]:
opt = Opt(:LD_SLSQP, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-Inf, 0.]) # forces `x` to have a non-negative value
min_objective!(opt, fg!) # specifies that optimization problem is on minimization
xtol_rel!(opt,1e-4) # set a lower relative xtol for convergence criteria

Similarly for constraint, where `constraint_f(x) <= 0` is imposed for all `x` 

In [21]:
function constraint_fg!(x::Vector, grad::Vector, a, b)
    if length(grad) > 0 # gradient of constraint_f(x)
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2] # constraint_f(x); constraint_f(x) <= 0 is imposed
end

constraint_fg! (generic function with 1 method)

Here, `a` and `b` are added to allow variants of `constraint_f(x)` in a handy way. For instance, to impose
```julia
(2*x[1] + 0)^3 - x[2] <= 0
```
AND
```julia
(-1*x[1] + 1)^3 - x[2] <= 0
```
one can simply run the following two lines:

In [22]:
inequality_constraint!(opt, (x,g) -> constraint_fg!(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> constraint_fg!(x,g,-1,1), 1e-8)

Ready to roll out:

In [29]:
(minf,minx,ret) = @btime optimize(opt, [1.234, 5.678])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  143.179 μs (206 allocations: 7.72 KiB)
got 0.5443310476210945 at [0.333333, 0.296296] after 11 iterations (returned XTOL_REACHED)


## Using `NLopt` without analytic formulae for gradients

Suppose that `f` you want to optimize is written in a fairly complex form and you do not have an access to the analytic formula of the gradient of `f`. Here is a solution using automatic differentiation:

### Automatic differentiation

In [24]:
# compute gradient by forward automatic differentiation
function g!(G::Vector, x::Vector)
    ForwardDiff.gradient!(G, f, x)
end

function fg!(x::Vector, grad::Vector)
    if length(grad) > 0 # gradient of f(x)
        g!(grad, x)
    end
    f(x)
end

fg! (generic function with 1 method)

Solve:

In [25]:
# define the optimization problem
opt = Opt(:LD_LBFGS, 2) # 2 indicates the length of `x`
lower_bounds!(opt, [-1.0, -1.0]) # find `x` above -2.0
upper_bounds!(opt, [2.0, 2.0]) # find `x` below 2.0
min_objective!(opt, fg!) # specifies that optimization problem is on minimization

# solve the optimization problem
(minf,minx,ret) = @btime optimize(opt, [1.0, 1.0])
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  6.604 ms (13 allocations: 720 bytes)
got 0.0 at [1.0, 1.0] after 1 iterations (returned FORCED_STOP)


## Integration with `NLSolversBase` interface

Add the following definition for `NLoptAdapter` to exploit the native support for numerical derivatives and autodifferentiation from `NLSolversBase`:

In [26]:
struct NLoptAdapter{T} <: Function where T <: AbstractObjective
    nlsolver_base::T
end

# note that the order is reversed
(adapter::NLoptAdapter)(x, df) = adapter.nlsolver_base.fdf(df, x) 

function NLoptAdapter(f, x, autodiff = :forward)
    return NLoptAdapter(OnceDifferentiable(f, x, autodiff = autodiff))
end

NLoptAdapter

Let's roll out:

In [27]:
myfunc(x) = sqrt(x[2])
x0 = [1.234, 5.678]
function myconstraint(x, a, b) 
    (a*x[1] + b)^3 - x[2]
end

# define objective and constraint, using NLoptAdapter
f_opt = NLoptAdapter(myfunc, x0)
c_1_opt = NLoptAdapter(x -> myconstraint(x,2,0), x0)
c_2_opt = NLoptAdapter(x -> myconstraint(x,-1,1), x0)

# define the optimization problem
opt = Opt(:LD_MMA, 2)
lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt,1e-4)

min_objective!(opt, f_opt)
inequality_constraint!(opt, c_1_opt, 1e-8)
inequality_constraint!(opt, c_2_opt, 1e-8)

# solve
(minf,minx,ret) = @btime optimize(opt, x0)
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  144.411 μs (271 allocations: 9.17 KiB)
got 0.5443310477213124 at [0.333333, 0.296296] after 11 iterations (returned XTOL_REACHED)


Works for central-difference methods too:

In [28]:
# define objective and constraint, using NLoptAdapter
f_opt = NLoptAdapter(myfunc, x0, :central)
c_1_opt = NLoptAdapter(x -> myconstraint(x,2,0), x0, :central)
c_2_opt = NLoptAdapter(x -> myconstraint(x,-1,1), x0, :central)

# define the optimization problem
opt = Opt(:LD_MMA, 2)
lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt,1e-4)

min_objective!(opt, f_opt)
inequality_constraint!(opt, c_1_opt, 1e-8)
inequality_constraint!(opt, c_2_opt, 1e-8)

# solve
(minf,minx,ret) = @btime optimize(opt, x0)
numevals = opt.numevals # the number of function evaluations
println("got $minf at $minx after $numevals iterations (returned $ret)")

  143.179 μs (205 allocations: 7.63 KiB)
got 0.5443310476210945 at [0.333333, 0.296296] after 11 iterations (returned XTOL_REACHED)
